# 8. Load the MNIST data and split it into a training set, a validation set, and a test set (e.g., use 50k instances for training, 10k for validation and 10k for testing)

In [6]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
# Load the MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.target = mnist.target.astype(np.uint8)

In [9]:
# Split the data
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(mnist.data, mnist.target, test_size=10000, random_state=42)

In [12]:
X_test.shape

(10000, 784)

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)

In [17]:
print("X_train: ", X_train.shape)
print("X_val: ",X_val.shape)
print("X_test: ",X_test.shape)

X_train:  (50000, 784)
X_val:  (10000, 784)
X_test:  (10000, 784)


In [19]:
# Exercise: Train various classifiers, such as Random Forest Classifiers, an Extra-Trees and an SVM
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier


In [20]:
# Declare the classifiers
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(max_iter=100, tol=20, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [21]:
# Training the models
estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
    print("Training the ",estimator)
    estimator.fit(X_train,y_train)

Training the  RandomForestClassifier(random_state=42)
Training the  ExtraTreesClassifier(random_state=42)
Training the  LinearSVC(max_iter=100, random_state=42, tol=20)
Training the  MLPClassifier(random_state=42)


In [22]:
for estimator in estimators:
    print("Score for ", estimator, "was ", estimator.score(X_val,y_val))

Score for  RandomForestClassifier(random_state=42) was  0.9692
Score for  ExtraTreesClassifier(random_state=42) was  0.9715
Score for  LinearSVC(max_iter=100, random_state=42, tol=20) was  0.859
Score for  MLPClassifier(random_state=42) was  0.9663


In [24]:
# The SVM classifier is outperformed by the others.

# Combine the classifiers in a voting classifier which outperforms all of them in the validation set

In [25]:
from sklearn.ensemble import VotingClassifier

In [26]:
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
    ("mlp_clf", mlp_clf),
]

In [27]:
voting_clf = VotingClassifier(named_estimators) ## Declare the voting classifier

In [28]:
voting_clf.fit(X_train, y_train) ## Fit the voting classifier  

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(random_state=42)),
                             ('svm_clf',
                              LinearSVC(max_iter=100, random_state=42, tol=20)),
                             ('mlp_clf', MLPClassifier(random_state=42))])

In [29]:
voting_clf.score(X_val, y_val)  ## Get the score

0.9713

In [30]:
# Compare the scores
for estimator in estimators:
    print("Score for ", estimator, "was ", estimator.score(X_val,y_val))
print("Score for the voting classifier: ",voting_clf.score(X_val, y_val))

Score for  RandomForestClassifier(random_state=42) was  0.9692
Score for  ExtraTreesClassifier(random_state=42) was  0.9715
Score for  LinearSVC(max_iter=100, random_state=42, tol=20) was  0.859
Score for  MLPClassifier(random_state=42) was  0.9663
Score for the voting classifier:  0.9713


The voting classifier does not outperform the best classifier

# Remove the SVM for the voting classifier

In [31]:
voting_clf.set_params(svm_clf=None)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(random_state=42)),
                             ('svm_clf', None),
                             ('mlp_clf', MLPClassifier(random_state=42))])

In [32]:
voting_clf.estimators

[('random_forest_clf', RandomForestClassifier(random_state=42)),
 ('extra_trees_clf', ExtraTreesClassifier(random_state=42)),
 ('svm_clf', None),
 ('mlp_clf', MLPClassifier(random_state=42))]

In [33]:
# Does not update the trained estimators
voting_clf.estimators_

[RandomForestClassifier(random_state=42),
 ExtraTreesClassifier(random_state=42),
 LinearSVC(max_iter=100, random_state=42, tol=20),
 MLPClassifier(random_state=42)]

In [34]:
# Delete the SVM classifier
del voting_clf.estimators_[2]

In [35]:
# obtain the score
voting_clf.score(X_val, y_val)

0.9742

In [36]:
# The SVM decreases the performance of the voting classifier

# Use a soft voting classifier 

In [37]:
voting_clf.voting = "soft"  # Not necessary to retrain the classifier, just set the voting parameter

In [38]:
# obtain the score
voting_clf.score(X_val, y_val)

0.9709

The hard voting classifier have a better performance than the soft voting

# Try on the test set

In [39]:
voting_clf.voting = "hard"
voting_clf.score(X_test, y_test)

0.9706

In [40]:
# For each classifier
[estimator.score(X_test, y_test) for estimator in voting_clf.estimators_]

[0.9645, 0.9691, 0.9636]

It is possible to see that the voting classifier outperforms all of its classifiers

# 9. Run the individual classifiers from the previous exercise to make predictions on the validation set, and create a new training set with the resulting predictions: each training instance is a vector containing the set of predictions from all your classifiers for an image, and the target is the image's class. Train a classifier on this new training set.

In [53]:
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)
print(X_val_predictions.shape)

# Run the individual classifiers 
for index, estimator in enumerate(estimators):
    print(index,estimator)
    # Make the predicitons on the validation set and create a vector containing the set of predictions from all classifiers
    X_val_predictions[:, index] = estimator.predict(X_val)


(10000, 4)
0 RandomForestClassifier(random_state=42)
1 ExtraTreesClassifier(random_state=42)
2 LinearSVC(max_iter=100, random_state=42, tol=20)
3 MLPClassifier(random_state=42)


In [54]:
X_val_predictions

array([[5., 5., 5., 5.],
       [8., 8., 8., 8.],
       [2., 2., 3., 2.],
       ...,
       [7., 7., 7., 7.],
       [6., 6., 6., 6.],
       [7., 7., 7., 7.]], dtype=float32)

In [58]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [59]:
rnd_forest_blender.oob_score_

0.9677

The blender can be used with MLPClassifiers, as example

Exercise: Evaluate the ensemble on the test set. For each image in the test set, make predictions with all your classifiers, then feed the predictions to the blender to get ensemble's predictions. How does it compare to the voting classifier your trained earlier?

In [60]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

In [64]:
# Run the individual classifiers 
for index, estimator in enumerate(estimators):
    print(index,estimator)
    # Make the predicitons on the validation set and create a vector containing the set of predictions from all classifiers
    X_test_predictions[:, index] = estimator.predict(X_test) ## Predict on the test set


0 RandomForestClassifier(random_state=42)
1 ExtraTreesClassifier(random_state=42)
2 LinearSVC(max_iter=100, random_state=42, tol=20)
3 MLPClassifier(random_state=42)


In [66]:
# Make the prediction
y_pred = rnd_forest_blender.predict(X_test_predictions)

In [67]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred,y_test)

0.9827

This stacking ensemble does not perform as well as the voting classifier we trained earlier, it's not quite as good as the best individual classifier.